In [ ]:
import parser_edsl as pe
import re

<Program> = <Entity> <Program> | <Eps>
<Entity> = enum <EntityBodyEnum> <EntityVar> | <RestType> <EntityBody> <EntityVar>
<EntityBodyEnum> = <EntityName> <Option1>
<Option1> = { <EnumList> } | <EnumListItem>
<EnumList> = <EnumListElem> <RestEnumList>
<RestEnumList> = , <EnumListElem> <RestEnumList> | <Eps>
<EnumListItem> = <Ident> <Option3>
<Option3> = = <Arith> | = <Expr> | <Eps>
<Expr> = sizeof(<Option2>)
<Types> = enum | <RestType>
<Option2> = <Types> <EntityName> | <TypeName>
<Arith> = <Arith> + <Term> | <Arith> - <Term> | <Term>
<Term> = <Term> * <Factor> | <Term> / <Factor> | <Factor>
<Factor> = <Number> | <Ident> | ( <Arith> )
<RestType> = struct | union
<TypeName> = char | int | double
<EntityName> = Varname_const
<Number> = Integer_const | Real_const
<Ident> = Varname_const
<EntityVar> = <EntityName> <RestNames>
<RestNames> = , <EntityName> <RestNames> | <Eps>l
<EntityBody> = <EntityName> <Option4>
<Option4> = { <List> } | <EnumListItem>
<List> = <ListItem> <RestList>
<RestList> = ; <ListItem> <RestList> | <Eps>
<ListItem> = <Entity> | <TypeName> <Ident>

In [ ]:
import abc
import enum
import typing
from dataclasses import dataclass


class Type_of_Entity(enum.Enum):
    Struct = 'struct'
    Enum = 'enum'
    Union = 'union'

class Type(enum.Enum):
    Integer = 'INTEGER'
    Double = 'DOUBLE'
    Char = 'CHAR'
    Float = 'FLOAT'

class Entity(abc.ABC):
    pass

class Expr(abc.ABC):
    pass

@dataclass
class Empty_Expr(Expr):
    pass

class Expr_Other(abc.ABC):
    pass

@dataclass
class Empty_Expr_Other(Expr_Other):
    pass

@dataclass
class Var_Expr_Other(Expr_Other):
    type: Type or Type_of_Entity
    value: list[str]

@dataclass
class Other_Body:
    name: str
    statements: list[Expr_Other]

@dataclass
class Other_Entity(Entity):
    type: Type_of_Entity
    body: Other_Body
    var: str

@dataclass
class Block_Expr_Other(Expr_Other):
    body: list[Other_Entity]

@dataclass
class Var_Expr(Expr):
    varname: str

@dataclass
class BinOpExpr(Expr):
    left: Expr
    op: str
    right: Expr

@dataclass
class Const_Expr(Expr):
    value: typing.Any
    type: Type

@dataclass
class Statement():
    var_name: str
    expr: Expr

@dataclass
class Enum_Body:
    name: str
    statements: list[Statement]

@dataclass
class Enum_Entity(Entity):
    type: Type_of_Entity
    body: Enum_Body 
    var: str

    
@dataclass
class Programm:
    entities: list[Entity]





In [ ]:
INTEGER = pe.Terminal('INTEGER', '[0-9]+', int, priority=7)
REAL = pe.Terminal('REAL', '[0-9]+(\\.[0-9]*)?(e[-+]?[0-9]+)?', float)
VARNAME = pe.Terminal('VARNAME', '[A-Za-z][A-Za-z0-9][\*\[\]]*', str.upper)

def make_keyword(image):
    return pe.Terminal(image, image, lambda name: None,
                       re_flags=re.IGNORECASE, priority=10)

KW_ENUM, KW_SIZEOF, KW_STRUCT, KW_CHAR, KW_INTEGER, KW_DOUBLE = \
    map(make_keyword, 'enum sizeof struct char int double'.split())

KW_UNION, KW_FLOAT = map(make_keyword, 'union float'.split())

NProgram, NEntity, NEntityBodyEnum, NEnumList, NRestEnumList = \
    map(pe.NonTerminal, 'Program Entity EntityBodyEnum EnumList RestEnumList'.split())

NEnumListItem, NExpr, NArith, NTerm, NFactor, NRestType, NTypes = \
    map(pe.NonTerminal, 'EnumListItem Expr Arith Term Factor, RestType Types'.split())

NTypeName, NEntityName, NNumber, NIdent, NEntityVar, NOption1 = \
    map(pe.NonTerminal, 'TypeName EntityName Number Ident EntityVar Option1'.split())

NRestNames, NEntityBody, NList, NRestList, NListItem, NOption2 = \
    map(pe.NonTerminal, 'RestNames EntityBody List RestList ListItem Option2'.split())

NOption3, NOption4 = \
    map(pe.NonTerminal, 'Option3, Option4'.split())


NProgram |= NEntity, ';', NProgram, lambda ent, pr: pr + [ent]
NProgram |= lambda: []

NEntity |= KW_ENUM, NEntityBodyEnum, NEntityVar
NEntity |= NRestType, NEntityBody, NEntityVar

NEntityBodyEnum |= NEntityName, NOption1
NOption1 |= '{',NEnumList, '}'
NOption1 |= NEnumListItem, lambda eli: [eli]

NEnumList |= NEnumListItem, NRestEnumList, lambda eli, reli: reli + [eli]
NRestEnumList |= ',', NEnumListItem, NRestEnumList, lambda eli, reli: reli + [eli]
NRestEnumList |= lambda: []

NEnumListItem |= NIdent, '=', NArith, lambda t, r: Statement(t, r)
NEnumListItem |= NIdent, '=', NExpr, lambda t, r: Statement(t, r)
NEnumListItem |= NIdent


NExpr |= KW_SIZEOF, '(', NOption2, ')'

NTypes |= KW_ENUM, lambda: 'enum'
NTypes |= NRestType

NOption2 |= NTypes, NEntityName
NOption2 |= NTypeName

NArith |= NArith, '+', NTerm, lambda l, r: BinOpExpr(l, '+', r)
NArith |= NArith, '-', NTerm, lambda l, r: BinOpExpr(l, '-', r)
NArith |= NTerm

NTerm |= NTerm, '*', NFactor, lambda l, r: BinOpExpr(l, '*', r)
NTerm |= NTerm, '/', NFactor, lambda l, r: BinOpExpr(l, '/', r)
NTerm |= NFactor

NFactor |= NNumber
NFactor |= NIdent
NFactor |= '(', NArith, ')'

NRestType |= KW_STRUCT, lambda: 'struct'
NRestType |= KW_UNION, lambda: 'union'

NTypeName |= KW_CHAR, lambda: 'CHAR'
NTypeName |= KW_INTEGER, lambda: 'INTEGER'
NTypeName |= KW_DOUBLE, lambda: 'DOUBLE'

NEntityName |= VARNAME
NNumber |= INTEGER, lambda v: Const_Expr(v, Type.Integer)
NNumber |= REAL, lambda v: Const_Expr(v, Type.Float)

NIdent |= VARNAME

NEntityVar |= NEntityName, NRestNames
NRestNames |= ',', NEntityName, NRestNames
NRestNames |= lambda: []

NEntityBody |= NEntityName, NOption4
NOption4 |= '{',NList, '}'
NOption4 |= NEnumListItem

NList |= NListItem, NRestList, lambda li, rli: rli + [li]

NRestList |= ';', NListItem, NRestList, lambda li, rli: rli + [li]
NRestList |= lambda: []

NListItem |= NEntity
NListItem |= NTypeName, NIdent

In [ ]:
p = pe.Parser(NProgram)
assert p.is_lalr_one()

p.add_skipped_domain('\\s')
p.add_skipped_domain('(//.)|(/\\*.\\*/)')

In [ ]:
from pprint import pprint
import sys

for filename in sys.argv[1:]:
    try:
        with open(filename) as f:
            tree = p.parse(f.read())
            pprint(tree)
    except pe.Error as e:
        print(f'Ошибка {e.pos}: {e.message}')
    except Exception as e:
        print(e)